In [1]:
import pandas as pd
import plotly.express as px

In [16]:
org_data = pd.read_parquet("escooter_history_2022.parquet")

In [106]:
# holiday (float) -> boolean
# wokringday (float) -> boolean
# registered_customer (boolean) -> float
org_data_clean = org_data.copy()
org_data_clean.holiday = org_data_clean.holiday.astype(bool)
org_data_clean.workingday = org_data_clean.workingday.astype(bool)

In [128]:
df_rcus = org_data_clean.copy()
df_rcus = df_rcus.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "sum"})
df_rcus

,registered_customer
datetime,
2020-01-04,654
2020-01-05,670
2020-01-06,1229
2020-01-07,1454
2020-01-08,1518
...,...
2021-12-30,4024
2021-12-31,7629
2022-01-01,5293


In [126]:
px.line(df_rcus, x=df_rcus.index, y="registered_customer")